In [1]:
import numpy as np
import pandas as pd
import re

from IPython.display import display
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize  
from collections import Counter
import string

from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.utils import pprint
from konlpy.tag import Twitter

import os

from tqdm import tqdm
import ast
from math import log


from gensim.models import Word2Vec
import random

from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer





In [2]:
twitter = Twitter()

C:\Users\Samsung\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [3]:
file_list = os.listdir(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long")
file_list_xlsx = [f for f in file_list if f.endswith(".xlsx")]

In [4]:
types_ = ["_Adj.xlsx","_Adverb.xlsx","_All.xlsx","_ETC.xlsx","_Noun.xlsx","_Unknown.xlsx","_Verb.xlsx"]
Adj= [f for f in file_list_xlsx if f.endswith(types_[0])]
Adverb = [f for f in file_list_xlsx if f.endswith(types_[1])]
All = [f for f in file_list_xlsx if f.endswith(types_[2])]
ETC = [f for f in file_list_xlsx if f.endswith(types_[3])]
Noun = [f for f in file_list_xlsx if f.endswith(types_[4])]
Unknown = [f for f in file_list_xlsx if f.endswith(types_[5])]
Verb = [f for f in file_list_xlsx if f.endswith(types_[6])]

In [14]:
ta_ = file_list[1]
ta_

'맨 케어_Adj.xlsx'

In [5]:
def load_data(ta_):
    path_ = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\{}"
    data_ = pd.read_excel(path_.format(ta_),encoding='cp949').drop(columns="Unnamed: 0")
    return data_

def load_data_v2(ta_,path_):
    data_ = pd.read_excel(path_.format(ta_),encoding='cp949').drop(columns="Unnamed: 0")
    return data_



In [6]:
data_.head()

NameError: name 'data_' is not defined

# Topic Modeling

In [7]:
def topic_moding(k,documents):
    K=k
    random.seed(0)
    D = len(documents)
    def p_topic_given_document(topic, d, alpha=0.1):
        return ((document_topic_counts[d][topic] + alpha) /(document_lengths[d] + K * alpha))

    def p_word_given_topic(word, topic, beta=0.1):
        return ((topic_word_counts[topic][word] + beta) /(topic_counts[topic] + V * beta))

    def topic_weight(d, word, k):
        return p_word_given_topic(word, k) * p_topic_given_document(k, d)

    def choose_new_topic(d, word):
        return sample_from([topic_weight(d, word, k) for k in range(K)])

    def sample_from(weights):
        total = sum(weights)
        rnd = total * random.random()
        for i, w in enumerate(weights):
            rnd -= w
            if rnd <= 0:
                return i

    document_topics = [[random.randrange(K) for word in document] for document in documents]
    document_topic_counts = [Counter() for _ in documents]
    topic_word_counts = [Counter() for _ in range(K)]
    topic_counts = [0 for _ in range(K)]
    document_lengths = list(map(len, documents))
    distinct_words = set(word for document in documents for word in document)
    V = len(distinct_words)

    D = len(documents)
    for d in range(D):
        for word, topic in zip(documents[d], document_topics[d]):
            document_topic_counts[d][topic] += 1
            topic_word_counts[topic][word] += 1
            topic_counts[topic] += 1


    for d in range(D):
        for word, topic in zip(documents[d], document_topics[d]):
            document_topic_counts[d][topic] += 1
            topic_word_counts[topic][word] += 1
            topic_counts[topic] += 1
    return document_topic_counts, topic_word_counts

In [8]:
data_.loc[5,"setence_tag"]

NameError: name 'data_' is not defined

In [97]:
for i2 in range(len(file_list_xlsx)):
    ta_ = file_list_xlsx[i2]
    read_path_ = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\{}"
    save_path_ = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_topic\LDA_{}"
    data_ = pd.read_excel(read_path_.format(ta_),encoding='cp949').drop(columns="Unnamed: 0")
    k = 1
    window = 8
    Topic_2 = []
    for num_ in data_.index:
        list_ = ast.literal_eval(data_.loc[num_,"setence_tag"])
        word_ = data_.loc[num_,"word"]
        #print(word_)
        try:
            removal = [li.remove(word_) for li in list_]
        except:
            pass
        docu_,topic_words = topic_moding(k,list_)
        top = topic_words[0].most_common(window)
        Topic_2.append(top)
    data_["Topic"] = Topic_2
    data_.to_excel(save_path_.format(ta_))

In [9]:
def topic_modeling_edit():
    for i2 in range(len(file_list_xlsx)):
        ta_ = file_list_xlsx[i2]
        read_path_ = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\{}"
        save_path_ = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_topic\LDA_{}"
        data_ = pd.read_excel(read_path_.format(ta_),encoding='cp949').drop(columns="Unnamed: 0")
        k = 1
        window = 8
        Topic_2 = []
        for num_ in data_.index:
            list_ = ast.literal_eval(data_.loc[num_,"setence_tag"])
            word_ = data_.loc[num_,"word"]
            #print(word_)
            try:
                removal = [li.remove(word_) for li in list_]
            except:
                pass
            docu_,topic_words = topic_moding(k,list_)
            top = topic_words[0].most_common(window)
            Topic_2.append(top)
        data_["Topic"] = Topic_2
        data_.to_excel(save_path_.format(ta_))

# TF IDF

In [10]:
tf_dic = {
    "Adj":Adj,
    "Adverb":Adverb,
    "All":All,
    "ETC":ETC,
    "Noun":Noun,
    "Unknown":Unknown,
    "Verb":Verb,
}

In [11]:
Adj

['맨 케어_Adj.xlsx',
 '베이스 메이크업_Adj.xlsx',
 '보디 케어_Adj.xlsx',
 '선 케어_Adj.xlsx',
 '스킨 케어_Adj.xlsx',
 '스타일러_Adj.xlsx',
 '이너 뷰티_Adj.xlsx',
 '컬러 메이크업_Adj.xlsx',
 '프래그런스_Adj.xlsx',
 '헤어 케어_Adj.xlsx']

In [12]:
key_ = list(tf_dic.keys())[0]
key_

'Adj'

In [13]:
tf_dic[key_]

['맨 케어_Adj.xlsx',
 '베이스 메이크업_Adj.xlsx',
 '보디 케어_Adj.xlsx',
 '선 케어_Adj.xlsx',
 '스킨 케어_Adj.xlsx',
 '스타일러_Adj.xlsx',
 '이너 뷰티_Adj.xlsx',
 '컬러 메이크업_Adj.xlsx',
 '프래그런스_Adj.xlsx',
 '헤어 케어_Adj.xlsx']

In [14]:
df_dic = {}
for file in tf_dic[key_]:
    #print(file)
    df_ = load_data(file).drop(columns=["setence_tag","setence","word2vec"])
    df_dic[file] = df_

In [15]:
list(df_dic.keys())

['맨 케어_Adj.xlsx',
 '베이스 메이크업_Adj.xlsx',
 '보디 케어_Adj.xlsx',
 '선 케어_Adj.xlsx',
 '스킨 케어_Adj.xlsx',
 '스타일러_Adj.xlsx',
 '이너 뷰티_Adj.xlsx',
 '컬러 메이크업_Adj.xlsx',
 '프래그런스_Adj.xlsx',
 '헤어 케어_Adj.xlsx']

In [137]:
d_1 = df_dic[key_2]
list(d_1["word"])

['촉촉',
 '산뜻',
 '깔끔',
 '민감',
 '예민',
 '시원',
 '간편',
 '강력',
 '칙칙',
 '간단',
 '은은',
 '답답',
 '탄탄',
 '저렴',
 '풍부',
 '궁금',
 '보송',
 '상쾌',
 '신선',
 '깨끗',
 '적당',
 '신기',
 '매끈',
 '자세',
 '충분',
 '찝찝',
 '푸석',
 '넉넉',
 '묵직',
 '비롯',
 '풍성',
 '말끔',
 '달콤',
 '화사',
 '웬만',
 '되직',
 '뿌듯',
 '익숙',
 '이만',
 '무방',
 '기특',
 '갑갑',
 '솔직',
 '강렬',
 '당연',
 '무난',
 '소중',
 '탁월',
 '활발',
 '어마어마',
 '훌륭',
 '쾌적',
 '용이',
 '든든',
 '꾸준',
 '빵빵',
 '흐뭇',
 '어둑',
 '도톰',
 '개운',
 '섬세',
 '지루',
 '비슷',
 '가뿐',
 '뽀송',
 '적절',
 '지만',
 '상세',
 '상이',
 '미지근',
 '만만',
 '마땅',
 '거뭇',
 '따뜻',
 '영롱',
 '중후',
 '혹독',
 '빠듯',
 '유연',
 '자잘',
 '빈번',
 '세세',
 '푸짐',
 '큼지막',
 '확실',
 '까칠',
 '까무잡잡',
 '따끈',
 '다채',
 '꿉꿉',
 '아찔',
 '복잡',
 '상큼',
 '빽빽',
 '지저분',
 '자글',
 '차분',
 '완연',
 '멀끔',
 '조밀',
 '딱딱',
 '죄송',
 '순박',
 '선선',
 '오묘',
 '생생',
 '깜찍']

In [139]:
i_3 = 0
key_2 = list(df_dic.keys())[i_3]
key_2

'맨 케어_Adj.xlsx'

In [149]:
save_path_2 = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_tf_idf\TFIDF_{}"
d_1 = df_dic[key_2]
TFIDF = []
for i_4 in d_1.index:
    t = d_1.loc[i_4,"word"]
    tf = d_1.loc[i_4,"count"]
    df = 0
    n = len(df_dic.keys())
    for key_3 in df_dic.keys():
        #print(key_3)
        d_2 = df_dic[key_3]
        if t in list(d_2["word"]):
            df += 1
        #print()
    #print(t,df)
    idf = log(n/(df + 1))
    tfidf = tf*idf
    TFIDF.append(tfidf)
d_1["TFIDF"] = TFIDF
d_1.to_excel(save_path_2.format(key_2))

## TF/IDF

In [157]:
path_ = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_topic\{}"


for i_2 in tqdm(range(len(tf_dic.keys()))):
    save_path_2 = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_tf_idf\TFIDF_{}"
    key_ = list(tf_dic.keys())[i_2]
    df_dic = {}
    for file in tf_dic[key_]:
        #print(file)
        df_ = load_data_v2(file).drop(columns=["setence_tag","setence","word2vec"])
        df_dic[file] = df_

    for i_3 in range(len(df_dic.keys())):
        key_2 = list(df_dic.keys())[i_3]
        #print(key_2)
        d_1 = df_dic[key_2]
        TFIDF = []
        for i_4 in d_1.index:
            t = d_1.loc[i_4,"word"]
            tf = d_1.loc[i_4,"count"]
            df = 0
            n = len(df_dic.keys())
            for key_3 in df_dic.keys():
                #print(key_3)
                d_2 = df_dic[key_3]
                if t in list(d_2["word"]):
                    df += 1
                #print()
            #print(t,df)
            idf = log(n/(df + 1))
            tfidf = tf*idf
            TFIDF.append(tfidf)
        d_1["TFIDF"] = TFIDF
        d_1.to_excel(save_path_2.format(key_2))


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:12<00:00, 10.31s/it]


## TFIDF 덮어 쓰기

- 값이 0이 나온것: idf = (log(10/9 +1) 일때

In [16]:
dir_path_ = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_topic\LDA_{}"
save_path_2 = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\data\result\result_long\most_long\most_tf_idf\TFIDF_{}"

for i_2 in tqdm(range(len(tf_dic.keys()))):
    key_ = list(tf_dic.keys())[i_2]
    df_dic = {}
    for file in tf_dic[key_]:
        #print(file)
        df_ = load_data_v2(file,dir_path_)#.drop(columns=["setence_tag","setence","word2vec"])
        df_dic[file] = df_

    for i_3 in range(len(df_dic.keys())):
        key_2 = list(df_dic.keys())[i_3]
        #print(key_2)
        d_1 = df_dic[key_2]
        TFIDF = []
        IDF = []
        DF = []
        for i_4 in d_1.index:
            t = d_1.loc[i_4,"word"]
            tf = d_1.loc[i_4,"count"]
            df = 0
            n = len(df_dic.keys())
            for key_3 in df_dic.keys():
                #print(key_3)
                d_2 = df_dic[key_3]
                if t in list(d_2["word"]):
                    df += 1
                #print()
            #print(t,df)
            idf = log(n/(df + 1))
            tfidf = tf*idf
            TFIDF.append(tfidf)
            IDF.append(idf)
            DF.append(df)
        d_1["TFIDF"] = TFIDF
        d_1.to_excel(save_path_2.format(key_2))

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [03:37<00:00, 31.01s/it]


In [176]:
save_path_2 = r"C:\Users\Samsung\Desktop\Project\TFIDF_{}"


In [182]:
key_ = list(tf_dic.keys())[4]
key_

'Noun'

In [183]:

df_dic = {}
for file in tf_dic[key_]:
    #print(file)
    df_ = load_data_v2(file,dir_path_)#.drop(columns=["setence_tag","setence","word2vec"])
    df_dic[file] = df_

for i_3 in range(len(df_dic.keys())):
    key_2 = list(df_dic.keys())[i_3]
    #print(key_2)
    d_1 = df_dic[key_2]
    TFIDF = []
    IDF = []
    DF = []
    for i_4 in d_1.index:
        t = d_1.loc[i_4,"word"]
        tf = d_1.loc[i_4,"count"]
        df = 0
        n = len(df_dic.keys())
        for key_3 in df_dic.keys():
            #print(key_3)
            d_2 = df_dic[key_3]
            if t in list(d_2["word"]):
                df += 1
            #print()
        #print(t,df)
        idf = log(n/(df + 1))
        tfidf = tf*idf
        TFIDF.append(tfidf)
        IDF.append(idf)
        DF.append(df)
    d_1["TFIDF"] = TFIDF
    d_1["IDF"] = IDF
    d_1["DF"] = DF
    d_1.to_excel(save_path_2.format(key_2))